In [50]:
# Importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from fastapi import FastAPI, HTTPException
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import ydata_profiling
from ydata_profiling import ProfileReport
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Cargar datos
archivo_parquet = 'C:\\Users\\felip\\Desktop\\Proyecto1\\Data\\df_movies.parquet'
df_movies = pd.read_parquet(archivo_parquet)
                            

In [4]:
# Examinar la información del DataFrame, como el tipo de datos y los valores nulos.
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49822 entries, 0 to 49821
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  5169 non-null   object        
 1   budget                 49822 non-null  float64       
 2   genres                 49822 non-null  object        
 3   id                     49822 non-null  object        
 4   original_language      49807 non-null  object        
 5   overview               48819 non-null  object        
 6   popularity             49822 non-null  float64       
 7   production_companies   49822 non-null  object        
 8   production_countries   49822 non-null  object        
 9   release_date           45359 non-null  datetime64[ns]
 10  revenue                49822 non-null  float64       
 11  runtime                49562 non-null  float64       
 12  spoken_languages       49822 non-null  object        
 13  s

In [5]:
# Reducimos el dataframe para trabajar la recomendación
df_movies_reduce = df_movies[df_movies['vote_count'] > 200]
df_movies_reduce.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4380 entries, 0 to 49536
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  1320 non-null   object        
 1   budget                 4380 non-null   float64       
 2   genres                 4380 non-null   object        
 3   id                     4380 non-null   object        
 4   original_language      4380 non-null   object        
 5   overview               4375 non-null   object        
 6   popularity             4380 non-null   float64       
 7   production_companies   4380 non-null   object        
 8   production_countries   4380 non-null   object        
 9   release_date           4264 non-null   datetime64[ns]
 10  revenue                4380 non-null   float64       
 11  runtime                4380 non-null   float64       
 12  spoken_languages       4380 non-null   object        
 13  status 

In [6]:
df_movies_reduce.isnull().sum()

belongs_to_collection    3060
budget                      0
genres                      0
id                          0
original_language           0
overview                    5
popularity                  0
production_companies        0
production_countries        0
release_date              116
revenue                     0
runtime                     0
spoken_languages            0
status                      0
tagline                   396
title                       0
vote_average                0
vote_count                  0
return                      0
release_year                0
crewjob                     0
director                    0
actors                      0
dtype: int64

In [7]:
# Observar la cantidad de valores unicos de cada columna
df_movies_reduce.nunique()

belongs_to_collection     556
budget                    420
genres                   1088
id                       3969
original_language          26
overview                 3967
popularity               3970
production_companies     3302
production_countries      419
release_date             3507
revenue                  3338
runtime                   153
spoken_languages          518
status                      3
tagline                  3609
title                    3895
vote_average               55
vote_count               1619
return                   3379
release_year               93
crewjob                     1
director                 1830
actors                   3968
dtype: int64

In [175]:
# Analisis del dataset - trabajos a realizar para lograr una recomendación basada en cadenas de strings:
# Campo 'status': trabajar sólo con los registros 'released'.
# Campo 'overview': Eliminar N/A
# Campo 'genres': desanidar para obtener sólo generos como una lista.
# Campo 'production_companies': desanidar y obtener sólo el nombre de la compania productora.
# Campo 'director', se agrupará en aquellos films con multiples directores.
# Tampoco se consideran necesarios los siguientes campos: 'belongs_to_collection', 'budget', 'popularity', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'vote_average', 'vote_count', 'return', 'release_year', 'crewjob', 'tagline', ni el campo 'status' ya que se trabajará sólo con los films estranados como se comentara previamente. 


In [8]:
# Seleccionamos sólo las peliculas con lanzamiento.
df_movies_released = df_movies_reduce[df_movies['status'] == 'Released']
df_movies_released.shape

(4378, 23)

In [9]:
# Elimina las filas con valores nulos en 'overview'
df_movies_released.dropna(axis=0, subset=['overview'], inplace=True)
df_movies_released.shape

(4373, 23)

In [10]:
# Seleccionar columnas mas relevantes
df_user = df_movies_released[['id','genres','original_language','overview','production_companies','title', 'director', 'actors']]
df_user.shape

(4373, 8)

In [11]:
df_director = df_user[['id', 'director']]
df_director.info

<bound method DataFrame.info of            id         director
0         862    John Lasseter
1        8844     Joe Johnston
5         949     Michael Mann
9         710  Martin Campbell
11      12110       Mel Brooks
...       ...              ...
49252  269795      Paul Currie
49313  353491    Nikolaj Arcel
49459  378236  Anthony Leondis
49496  339692  Ric Roman Waugh
49536  417870   Malcolm D. Lee

[4373 rows x 2 columns]>

In [12]:
# Reagrupamos el dataframe para obtener sólo un id con una lista de directores
df_director_reagrupado = df_director.groupby('id')['director'].apply(list).reset_index()
df_director_reagrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3965 entries, 0 to 3964
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3965 non-null   object
 1   director  3965 non-null   object
dtypes: object(2)
memory usage: 62.1+ KB


In [13]:
df_user_dir = df_user.merge(df_director_reagrupado, on='id', how='left')

In [14]:
df_user_dir.drop(columns=['director_x'], inplace=True)

In [ ]:
df_user_dir.rename(columns={'director_y': 'directors'}, inplace=True)
 
df_user_dir


In [90]:
df_user_dir.drop_duplicates(subset=['overview'], inplace=True)  #df = df.drop_duplicates(subset=['nombre_columna'])
df_user_dir.dropna()
df_user_dir.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3965 entries, 0 to 4372
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    3965 non-null   object
 1   genres                3965 non-null   object
 2   original_language     3965 non-null   object
 3   overview              3965 non-null   object
 4   production_companies  3965 non-null   object
 5   title                 3965 non-null   object
 6   actors                3965 non-null   object
 7   directors             3965 non-null   object
dtypes: object(8)
memory usage: 278.8+ KB


In [16]:
# Expandimos las columnas 'genres' y 'production_companies' para limpiar la informacion innecesaria

def expandcolumns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.
        # Expande la columna column en filas, de modo que cada elemento de la lista de diccionarios se convierte en una fila separada en df
        df = df.explode(column)
        # Normaliza la columna column después de expandirla, lo que significa que convierte cada diccionario dentro de la lista en una serie de columnas.
        col_df = pd.json_normalize(df[column])
        # Renombra las columnas normalizadas agregando el nombre original de la columna como prefijo, para evitar conflictos si hay nombres de columnas duplicados en diferentes listas de diccionarios.
        col_df = col_df.add_prefix(f'{column}')
        # Elimina la columna original column del DataFrame df y la reemplaza con las columnas normalizadas (col_df), asegurándose de que el índice del DataFrame se reinicie para evitar problemas con el índice de las filas.
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['genres']
columns_to_expand2 = ['production_companies']

#Expandir las columnas de listas de diccionarios
df_genres = expandcolumns(df_user_dir, columns_to_expand)
df_companies = expandcolumns(df_user_dir, columns_to_expand2)

In [17]:
# Seleccionamos sólo las columnas genresname y id
df_genres = df_genres[['id', 'genresname']]
df_genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11714 entries, 0 to 11713
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          11714 non-null  object
 1   genresname  11713 non-null  object
dtypes: object(2)
memory usage: 183.2+ KB


In [18]:
# Seleccionamos sólo las columnas genresname y id
df_companies = df_companies[['id', 'production_companiesname']]
df_companies.info()
                            

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14636 entries, 0 to 14635
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        14636 non-null  object
 1   production_companiesname  14626 non-null  object
dtypes: object(2)
memory usage: 228.8+ KB


In [19]:
# Reagrupamos el dataframe para obtener sólo un id con una lista de generos
df_genres_reagrupado = df_genres.groupby('id')['genresname'].apply(list).reset_index()
df_genres_reagrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3965 entries, 0 to 3964
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          3965 non-null   object
 1   genresname  3965 non-null   object
dtypes: object(2)
memory usage: 62.1+ KB


In [20]:
# Reagrupamos el dataframe productoras para obtener sólo un id con una lista de generos
df_companies_reagrupado = df_companies.groupby('id')['production_companiesname'].apply(list).reset_index()
df_companies_reagrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3965 entries, 0 to 3964
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        3965 non-null   object
 1   production_companiesname  3965 non-null   object
dtypes: object(2)
memory usage: 62.1+ KB


In [21]:
df_user_dir = df_user_dir.merge(df_genres_reagrupado, on='id', how='left')

In [22]:
# Eliminamos columna antigua de genero
df_user_dir.drop(columns=['genres'], inplace=True)

In [23]:
df_recommend = df_user_dir.merge(df_companies_reagrupado, on='id', how='left')

In [24]:
# Eliminamos columna antigua de productoras
df_recommend.drop(columns=['production_companies'], inplace=True)

In [25]:
# Renombraos columnas
df_recommend.rename(columns={'genresname': 'genres', 'production_companiesname': 'production_companies'}, inplace=True)

In [26]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\felip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [111]:
# stop_words = set(stopwords.words('english'))
# stop_words_es = set(stopwords.words('spanish'))


In [27]:
df_recommend['title'] = df_recommend['title'].astype(str)


In [28]:
df_recommend.to_parquet('C:\\Users\\felip\\Desktop\\Proyecto1\\Data\\df_recommend.parquet')

In [129]:
# Creamos una funcion para limpiar y tokenizar nuestras columnas

# def clear_text(text):
#     tokens = word_tokenize(text)
#     tokens = [word.lower() for word in tokens if word.isalpha()] # Elimina signos de puntuacion y llevamos a minuscula
#     tokens = [word for word in tokens if word not in stop_words and word not in stop_words_es] # Se filtra por stopwords
#     return tokens

In [ ]:
# Limpiamos los registros complejos

# df_recommend['final_title'] = df_recommend['title'].apply(lambda x: clear_text(x))
# df_recommend['final_overview'] = df_recommend['overview'].apply(lambda x: clear_text(x))

Visualizamos cantidad de votos

In [196]:
# plt.figure(figsize=(10,6))

# sns.histplot(data=df_movies_sindirector, x='vote_count', bins=1238, kde= True, color='blue', label='Cantidad votos')

# #sns.histplot(data=df_movies_sindirector, x='vote_average', bins=10, kde= True, color='red', label='Promedio de votos')

# plt.title('Análisis de votos')
# plt.legend()
# plt.show()

In [34]:
# Convertimos columna 'title' a una representacón numérica usando Tf-IDF
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(df_recommend['title'] + ' ' + df_recommend['genres'].astype(str) + ' ' + df_recommend['actors'].astype(str) + ' ' + df_recommend['production_companies'].astype(str) + ' ' + df_recommend['overview'].astype(str) + ' ' + df_recommend['directors'].astype(str) + ' ' + df_recommend['genres'].astype(str) + ' ' + df_recommend['genres'].astype(str)) #+ ' ' + df_movies_sindirector['actors']) #+ ' ' + df_movies_sindirector['genresname'])

# Añadimos las columnas numéricas a nuestra matriz.
# features = np.column_stack([matrix.toarray(), df_movies_sindirector['vote_count'], df_movies_sindirector['vote_average']])


In [31]:
# Convertimos columna 'title' a una representacón numérica usando Tf-IDF
# vectorizer = TfidfVectorizer()
# matrix = vectorizer.fit_transform(df_movies_sindirector['title'] + ' ' + df_movies_sindirector['actors']) #+ ' ' + df_movies_sindirector['actors']) #+ ' ' + df_movies_sindirector['genresname'])

# # Añadimos las columnas numéricas a nuestra matriz.
# features = np.column_stack([matrix.toarray(), df_movies_sindirector['vote_count'], df_movies_sindirector['vote_average']])


In [35]:
# Reindexamos el dataframe
df_recommend = df_recommend.reset_index(drop=True)

# Calculamos la matriz de similitud de coseno
cosine_matrix = cosine_similarity(matrix)

# # Elaboramos función para recomentar 5 peliculas similares a un titulo indicado
# nombre_pelicula = 'Top Gun'
# titulo = df_movies_sindirector[df_movies_sindirector['title'] == nombre_pelicula]

# if not titulo.empty:
#     product_index = titulo.index[0]
#     product_similarities = cosine_matrix[product_index]
#     most_similar_products_indices = np.argsort(-product_similarities)
#     top_5_indices = most_similar_products_indices[:5]  # Obtén índices para las 5 primeras películas
#     top_5_peliculas = df_movies_sindirector.loc[top_5_indices, 'title']
#     print(top_5_peliculas)  # Imprime solo los 5 títulos recomendados
#     # most_similar_products = df_movies_sindirector.loc[most_similar_products_indices, 'title']
#     # print(most_similar_products)



In [31]:
# def recomendacion(titulo):
    
#     titulo = titulo.lower()  # Convertimos todo a minúsculas
#     titulo_pelicula = df_recommend[df_recommend['title'].str.lower() == titulo]

#     if titulo_pelicula.empty:
#         print(f"No se encontró la película {titulo}")
        
#     indice_producto = titulo_pelicula.index[0]
#     similitudes_producto = cosine_matrix[indice_producto]
#     indices_top_5_similares = np.argsort(-similitudes_producto)[1:6]
#     top_5_peliculas = df_recommend.loc[indices_top_5_similares, 'title']
#     top_5_peliculas = top_5_peliculas.reset_index(drop=True)
#     print(f"Películas similares a {titulo.capitalize()}:")
#     for i, pelicula in enumerate(top_5_peliculas, start=1):
#         print(f"{i}. {pelicula}")
        
   

In [48]:
def recomendacion(titulo: str):
    titulo = titulo.lower()
    titulo_pelicula = df_recommend[df_recommend['title'].str.lower() == titulo]

    if titulo_pelicula.empty:
        raise HTTPException(status_code=404, detail=f"No se encontró la película {titulo}")

    indice_producto = titulo_pelicula.index[0]
    similitudes_producto = cosine_matrix[indice_producto]
    indices_top_5_similares = np.argsort(-similitudes_producto)[1:6]
    top_5_peliculas = df_recommend.loc[indices_top_5_similares, 'title'].tolist()  # Convierte a lista
    return print(f"Películas similares a {titulo.capitalize()} son:{top_5_peliculas}")

In [49]:
# Ejemplo de uso
pelicula_recomendada = "titanic"
recomendacion(pelicula_recomendada)

Películas similares a Titanic son:['Les Misérables', 'Return of the Jedi', 'True Romance', 'The Departed', 'Master and Commander: The Far Side of the World']
